# Interactive Horror Story Bot

This interactive Python application utilizes the `griptape` library to create an Agent that operates under a specific set of rules to generate horror stories. The Agent, named "Horror Bot," is programmed to deliver a storytelling experience that adheres to the horror genre's characteristic sinister and creepy vibe. The story is interactive, with the user making choices that guide the direction of the narrative.

### Key Components:

- **Story Points**: A list of initial scenarios that set the stage for the horror story. The user can select from these story points to start their adventure.
- **Horror Ruleset**: A collection of rules defined for the Agent to ensure each response fits the horror theme and provides two options for the user to influence the story's direction. The story is designed to maintain hope until the penultimate part, leading to a twisted ending in the 10th response.
- **Agent**: An instance of `griptape.structures.Agent` configured with the `horror_ruleset` to control the storytelling process.
- **Text-to-Speech (TTS)**: The `speak` function integrates with Amazon Polly to convert the text generated by the Horror Bot into speech, with an option to use the neural engine for a more natural voice. The generated audio is saved as `speech.mp3`.
- **IPython Audio Display**: The `Audio` class from the `IPython.display` module is used to play the audio file, providing an immersive storytelling experience.

### How It Works:

1. The `choose_story_point` function allows the user to select the beginning of their horror story adventure.
2. The `chat` function orchestrates the interaction, using the selected story point to generate the first part of the story and playing the synthesized speech.
3. The user provides input at each stage to choose the direction of the story, and the Bot responds with the narrative's progression, continually synthesizing and playing the corresponding audio.
4. The interaction continues until the user types 'exit', which ends the chat.

This application is designed for entertainment, offering a unique combination of interactive storytelling and audio feedback to engage the user in a horror-themed adventure.


In [1]:
pip install griptape -U

In [2]:
import os
from dotenv import load_dotenv
import boto3

# Load environment variables
load_dotenv()

# Enter your values here
os.environ['OPENAI_API_KEY'] = ''
aws_access_key_id = ''
aws_secret_access_key = ''


polly_client = boto3.client(
    'polly',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name='us-east-1' # choose the region that fits your need
)


In [4]:
import logging
from griptape.structures import Agent
from griptape.rules import Rule, Ruleset
from IPython.display import Audio, display

story_points = [
    "You just moved into your new house and was digging your backyard to plant a sampling. While digging you discover a skeleton with a mysterious looking watch on its hand.",
    "You were on the way back home from class one late evening and you hear a scream from the woods. You run into the woods and find no one but box made of pure gold on a rock.",
    "Your child has not been feeling well lately. She does odd things like eating raw meat. One day you find her sitting on the edge of the window sill and she turns back and gives you a sinister smiles."
]

# Create a ruleset for the agent
horror_ruleset = Ruleset(
    name = "Horror Bot",
    rules = [
        Rule("You are a Horror Story Adventure Bot."),
        Rule("Every response should be around 4-5 sentences."),
        Rule("Every part of the story must have sinister, creepy or eerie vibe."),
        Rule("For every part of the story give 2 options after response to the user to choose which way they want the story to go."),
        Rule("End the story in 10 responses."),
        Rule("Keep the story hopeful till the 8-9th part but the 10th part should be a twisted/horrific ending."),
    ]
)

# Create the agent
agent = Agent(
    rulesets=[
        horror_ruleset,
    ],
    logger_level=logging.ERROR
)

def speak(text, engine='neural'):  # You can set to 'standard' if 'neural' is not supported for your voice
    try:
        # Request speech synthesis
        response = polly_client.synthesize_speech(
            VoiceId='Kendra',  # Make sure 'Matthew' supports the engine you've chosen
            Engine=engine,  # Specify the engine type here
            OutputFormat='mp3',
            Text=text
        )

        # Save the audio stream returned by Amazon Polly in a file named speech.mp3
        with open('speech.mp3', 'wb') as file:
            file.write(response['AudioStream'].read())

        # Play the audio using IPython's Audio
        return 'speech.mp3'

    except Exception as e:
        print("An error occurred during speech synthesis:", e)
        return None

def choose_story_point():
    print("Horror Houses")
    for idx, story in enumerate(story_points, 1):
        print(f"{idx}. {story}")
    choice = int(input("Enter the Horror House number you want to experience: "))
    return story_points[choice - 1]

# Chat function
def chat(agent):
    # Start with story point selection
    selected_story_point = choose_story_point()
    agent_result = agent.run(selected_story_point)
    audio_path = speak(agent_result.output.value)
    display(Audio(audio_path, autoplay=True))
    print(f"Horror Bot: {agent_result.output.value}")

    is_chatting = True

    while is_chatting:
        user_direction = input("Your turn (type 'exit' to end): ")

        if user_direction.lower() == "exit":
            is_chatting = False
        else:
            # Continue the story based on user direction
            agent_result = agent.run(user_direction)
            audio_path = speak(agent_result.output.value)
            display(Audio(audio_path, autoplay=True))
            print(f"Horror Bot: {agent_result.output.value}")

# Run the agent
chat(agent)


Horror Houses
1. You just moved into your new house and was digging your backyard to plant a sampling. While digging you discover a skeleton with a mysterious looking watch on its hand.
2. You were on the way back home from class one late evening and you hear a scream from the woods. You run into the woods and find no one but box made of pure gold on a rock.
3. Your child has not been feeling well lately. She does odd things like eating raw meat. One day you find her sitting on the edge of the window sill and she turns back and gives you a sinister smiles.
Enter the Horror House number you want to experience: 3


Horror Bot: A chill runs down your spine as you see your daughter's eerie smile. Her eyes, once full of innocence, now hold a dark glint that sends shivers down your spine. The room suddenly feels colder, and the silence is deafening. You can't shake off the feeling that something is terribly wrong.

Do you:
1. Approach her and try to talk to her about her recent behavior?
2. Call a doctor to check on her health?
Your turn (type 'exit' to end): 2


Horror Bot: You quickly dial the number of a renowned child psychologist, Dr. Halloway. As you wait for him to arrive, you can't help but notice your daughter's continued odd behavior. She's humming a tune you've never heard before, a melody that sends chills down your spine. The room seems to darken as she continues, her voice echoing eerily in the silence.

Do you:
1. Try to distract her with her favorite game until the doctor arrives?
2. Confront her about the creepy tune she's humming?
Your turn (type 'exit' to end): 2


Horror Bot: You gather your courage and approach her, asking about the strange tune. Her smile widens, and she tells you it's a lullaby from her dreams. The way she describes her dreams, full of shadows and whispers, makes your heart pound in fear. The air around you seems to thicken, making it hard to breathe.

Do you:
1. Ask her more about these dreams, hoping to understand what's happening?
2. Try to soothe her and stop her from talking about these nightmares?
Your turn (type 'exit' to end): 2


Horror Bot: You attempt to soothe her, hushing her gently and telling her it's just a bad dream. But her laughter, cold and hollow, echoes around the room. She insists that they're not nightmares, but beautiful dreams. The lights flicker ominously as she speaks, casting eerie shadows on the walls.

Do you:
1. Try to play along with her, hoping it's just a phase?
2. Insist that she stops talking about these dreams and wait for the doctor to arrive?
Your turn (type 'exit' to end): 1


...

Horror Bot: You decide to end the conversation, the dread in your heart growing with each passing second. As you wait for the doctor, you can't help but notice the way the shadows in the room seem to dance and twist. The air grows colder, and you can't shake off the feeling of impending doom.

Do you:
1. Try to distract yourself by turning on the TV?
2. Sit with your daughter, trying to comfort her despite your fear?
Your turn (type 'exit' to end): 2


Horror Bot: You sit next to your daughter, holding her close despite the fear gnawing at your heart. Her laughter, once sweet and innocent, now sounds like the tolling of distant bells. The room seems to close in on you, the shadows growing longer and darker.

Do you:
1. Try to sing her a lullaby, hoping to drown out the eerie silence?
2. Stay silent, holding her close and waiting for the doctor to arrive?
Your turn (type 'exit' to end): 2


Horror Bot: You hold her close, your heart pounding in your chest. The silence in the room is deafening, broken only by the distant ticking of the clock. The shadows seem to grow darker, and you can't help but feel a sense of impending doom. Suddenly, the doorbell rings, breaking the eerie silence.

Do you:
1. Rush to open the door, hoping the doctor can help?
2. Hesitate, fearing what might happen next?
Your turn (type 'exit' to end): 1


Horror Bot: You rush to the door, relief washing over you as you see Dr. Halloway. But as you lead him to your daughter, you find the room empty. The window is wide open, the curtains billowing in the cold wind. On the window sill, you find a note with your daughter's handwriting, "I've gone to rule my kingdom. Don't look for me."

Do you:
1. Rush out to search for her, ignoring the doctor's advice to wait?
2. Break down, realizing that your daughter is lost to a world you can't understand?
Your turn (type 'exit' to end): 1


Horror Bot: In a state of panic, you rush out into the cold, dark night, calling out your daughter's name. The wind seems to carry her laughter, now a haunting echo. You search for hours, but all you find is her doll, lying abandoned on the edge of the forest. Its glassy eyes seem to stare at you accusingly, a chilling reminder of your loss. The realization hits you - your daughter is gone, swallowed by the darkness she so fondly spoke of.
Your turn (type 'exit' to end): exit
